In [1]:
%load_ext autoreload
%autoreload 2

Try classical SVM

In [2]:
#scikit learn and tools
import pandas as pd
import numpy as np;
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from embedding import kernel_matrix

In [3]:
#load dataset with panda
#data are scaled outside the notebook
env = pd.read_csv('env.sel2.scaled.csv')  

In [4]:
#take a look of the dataset
env.describe()

,illuminance,blinds,lamps,co,rh,co2,temp,occupancy
count,1.093400e+04,10934.000000,10934.000000,1.093400e+04,10934.000000,1.093400e+04,1.093400e+04,10934.000000
mean,2.859327e-17,0.803111,0.504677,-1.871559e-16,0.471725,1.455657e-16,-4.159021e-16,-0.315164
std,1.000000e+00,0.371107,0.498983,1.000000e+00,0.076689,1.000000e+00,1.000000e+00,0.949081
min,-9.310658e-01,0.000000,0.000000,-8.039283e-01,0.243300,-1.145063e+00,-1.711094e+00,-1.000000
25%,-8.848906e-01,1.000000,0.000000,-7.808253e-01,0.431300,-7.923978e-01,-8.078199e-01,-1.000000
50%,2.542024e-02,1.000000,1.000000,-2.725584e-01,0.487400,-3.964529e-01,-1.153100e-01,-1.000000
75%,3.882253e-01,1.000000,1.000000,3.743267e-01,0.519800,6.966369e-01,7.428010e-01,1.000000
max,9.814560e+00,1.000000,1.000000,6.542838e+00,0.633300,3.950372e+00,2.594513e+00,1.000000


In [5]:
#DEFINE design matrix
Y = env['occupancy']
#X = env[['illuminance', 'blinds','lamps','co', 'rh', 'co2', 'temp']]
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]

X.head()

,illuminance,blinds,lamps,rh,co2,temp
0,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
1,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
2,-0.90468,1.0,0.0,0.4069,-0.910870,-0.030001
3,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001
4,-0.90468,1.0,0.0,0.4069,-0.795139,-0.030001


In [6]:
#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [7]:
#sanity check
print('TRAIN*****')
display(X_train.head())
display(X_train.describe())
print('TEST*****')
display(X_test.head())
display(X_test.describe())

TRAIN*****


,illuminance,blinds,lamps,rh,co2,temp
3429,0.137560,0.04,1.0,0.4520,0.561767,1.134220
10512,-0.000966,1.00,1.0,0.5302,0.881625,-0.807820
3453,0.111174,0.04,1.0,0.4545,0.630572,0.958583
3457,0.111174,0.04,1.0,0.4545,0.630572,0.958583
6506,-0.898083,1.00,0.0,0.5283,-0.798322,-1.410003


,illuminance,blinds,lamps,rh,co2,temp
count,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000,8200.000000
mean,0.000985,0.803154,0.506168,0.471101,-0.004185,0.006651
std,0.995350,0.370942,0.498947,0.077109,0.995483,1.000817
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.884891,1.000000,0.000000,0.425800,-0.792398,-0.807820
50%,0.032017,1.000000,1.000000,0.486800,-0.396453,-0.115310
75%,0.388225,1.000000,1.000000,0.521600,0.672035,0.742801
max,9.814560,1.000000,1.000000,0.633300,3.950372,2.594513


TEST*****


,illuminance,blinds,lamps,rh,co2,temp
10898,-0.878294,1.00,0.0,0.4972,-1.120922,-1.450149
1012,3.811786,1.00,1.0,0.3831,-0.585087,1.003747
4212,-0.931066,0.14,0.0,0.4752,-0.304863,0.356400
8445,0.163946,1.00,1.0,0.5320,1.136333,0.742801
10720,-0.878294,1.00,0.0,0.4972,-1.120922,-1.450149


,illuminance,blinds,lamps,rh,co2,temp
count,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000,2734.000000
mean,-0.002953,0.802979,0.500205,0.473596,0.012552,-0.019948
std,1.013998,0.371669,0.499156,0.075396,1.013508,0.997461
min,-0.931066,0.000000,0.000000,0.243300,-1.145063,-1.711094
25%,-0.884891,1.000000,0.000000,0.437400,-0.793198,-0.807820
50%,0.025420,1.000000,0.500000,0.489300,-0.395991,-0.160473
75%,0.348647,1.000000,1.000000,0.519800,0.742202,0.742801
max,9.814560,1.000000,1.000000,0.633300,3.804566,2.594513


In [8]:
#WARNING: convert data to numpy. Quantum stuff (Qiskit) do not like PANDAS
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()


In [9]:
#define data for training
K = 150
K_test = 50

In [10]:
#try SVM using RBF kernel
svm = SVC(kernel='rbf').fit(X_train_np[:K], y_train_np[:K]);
#result...
predictions = svm.predict(X_test_np[:K_test])
accuracy_score(predictions, y_test[:K_test])

0.78

Try quantum Kernel Method

In [ ]:
#Create a cotainer
from embedding import CircuitContainer
from embedding import QEmbedding

c = CircuitContainer(f_embedding= QEmbedding.createQuantumEmbedding, nwire=6, obs=['ZIIIII', 'ZIIZZI'])

In [11]:
#try SVM using linear kernel (dot product)
svm_quantum = SVC(kernel=kernel_matrix).fit(X_train_np[:K], y_train_np[:K]);

*** Created quantum template for feature map using 1 qubit ***


CircuitError: 'duplicate qubit arguments'

In [57]:
q_predictions = svm_quantum.predict(X_test_np[:K_test])
accuracy_score(q_predictions, y_test_np[:K_test])

0.78